In [48]:
import pandas as pd
import numpy as np
import camelot

## NIGC Gaming Tribe Report 2025

In [61]:
tables_a = camelot.read_pdf('../data/TribesAlphabeticalList_NIGC_GamingTribeReport_2025.pdf',
                          flavor='stream',
                          row_tol=5,
                          pages='all')
report_a = tables_a[0].df
# report_a


In [62]:
tables_a_df_list = [table.df.iloc[3:, :] for table in tables_a]

In [64]:
report_df = pd.concat(tables_a_df_list, axis=0)
report_df

,0,1,2,3
3,Absentee Shawnee Tribe of Indians of OK,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Class Type: Class II & III See 25 CFR 502.3 & 4,Email Address: Governor@astribe.com
4,,Thunderbird Casino - Shawnee,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Game Class Type: Class II & III See 25 CFR 5...
5,,Work Phone: 405-360-9270,Work Fax: 405-573-3211,Email Address: LisaF@TBECInc.com
6,,Thunderbird Entertainment Center,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Game Class Type: Class II & III See 25 CFR 5...
7,,Work Phone: 405-360-9270,Work Fax: 405-360-9288,Email Address: LisaF@TBECInc.com
...,...,...,...,...
48,,Work Phone: 707-482-1777,Work Fax: 707-482-1772,Email Address: rbacon@yuroktribe.nsn.us
49,,Redwood Hotel Casino,"P.O. Box 1027 190 Klamath Blvd., Klamath CA 95548",Game Class Type: Class III See 25 CFR 502.3 & 4
50,,Work Phone: 707-482-1777,Work Fax: 707-482-1772,Email Address: rbacon@yuroktribe.nsn.us
51,,Redwood Hotel Casino,"P.O. Box 1027 190 Klamath Blvd., Klamath CA 95548",Game Class Type: Class III See 25 CFR 502.3 & 4


In [65]:
report_df.reset_index(drop=True, inplace=True)

In [ ]:
is_nonempty0 = (report_df[0]!='').to_numpy()+0
# Find all indices where there are two 1's in a row
# This indicates a place where a tribe/nation name spilled over to two rows.
name_bleeds_over0_idx = np.where((is_nonempty0[:-1] == 1) & (is_nonempty0[1:]==1))[0]
# name_bleeds_over0_idx
for i in name_bleeds_over0_idx:
    report_df.loc[i, 0] += ' ' + report_df.loc[i+1, 0]
    report_df.loc[i+1, 0] = '' # No longer need the bleed-over text, it has been absorbed into the full name above


array([  14,   32,   48,   51,   76,   88,   98,  103,  109,  131,  235,
        245,  327,  346,  359,  362,  372,  375,  383,  386,  389,  416,
        425,  446,  459,  470,  474,  491,  500,  504,  572,  600,  624,
        627,  650,  656,  666,  674,  686,  691,  718,  719,  720,  721,
        722,  723,  724,  725,  726,  727,  728,  729,  730,  731,  732,
        733,  734,  735,  736,  737,  738,  739,  740,  741,  742,  743,
        744,  745,  746,  747,  748,  749,  750,  751,  752,  753,  754,
        755,  756,  757,  758,  800,  806,  863,  893,  905,  930,  954,
       1015, 1024, 1037, 1044, 1047, 1059, 1080, 1085, 1096, 1103, 1150,
       1166, 1169, 1182, 1266, 1292, 1304, 1338, 1390, 1415, 1492, 1522])

The `row_tol` parameter does not help us here since the multi-line tribe/nation name has the same distance between lines as the tribe/nation address and first facility name do. We may as well just keep all of these lines separate and rejoin the tribe/nation names based on adjacency and presence of a newline character. We then just extract the two-letter state abbreviation from the address with regular expressions. Are there any cases of a tribe/nation owning a facility in another state's borders?

Steps
1. Remove all rows that have empty column 0 AND have a column 1 that contains a row containing "Work Phone"
2. Stamp the tribe/nation name onto each of their facilities' names.
3. Extend to all pages of the pdf, not just the first page.

In [68]:
# report_a[1][report_a[1].str.contains('Work Phone')]

In [69]:
report_df = report_df.loc[~report_df[1].str.contains('Work Phone'), :]

In [70]:
report_df_clean = report_df.replace('', np.nan)

In [71]:
is_facility_idx = report_df_clean[0].isna()

In [72]:
report_df_clean['is_facility'] = is_facility_idx

In [73]:
report_df_clean[0] = report_df_clean[0].ffill()

In [99]:
report_df_clean.reset_index(inplace=True, drop=True)

In [102]:
report_df_clean

,0,1,2,3,is_facility
0,Absentee Shawnee Tribe of Indians of OK,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Class Type: Class II & III See 25 CFR 502.3 & 4,Email Address: Governor@astribe.com,False
1,Absentee Shawnee Tribe of Indians of OK,Thunderbird Casino - Shawnee,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Game Class Type: Class II & III See 25 CFR 5...,True
2,Absentee Shawnee Tribe of Indians of OK,Thunderbird Entertainment Center,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Game Class Type: Class II & III See 25 CFR 5...,True
3,Agua Caliente Band of Cahuilla Indians,"5401 Dinah Shore Dr., Palm Springs CA 92264",Class Type: Class II & III See 25 CFR 502.3 & 4,Email Address: laviles@aguacaliente.net,False
4,Agua Caliente Band of Cahuilla Indians,Agua Caliente Casino Rancho Mirage,"5401 Dinah Shore Dr., Palm Springs CA 92264",Game Class Type: Class II & III See 25 CFR 5...,True
...,...,...,...,...,...
967,Yurok Tribe of the Yurok Reservation,"P.O. Box 1027 190 Klamath Blvd., Klamath CA 95548",Class Type: Class II & III See 25 CFR 502.3 & 4,Email Address: jjames@yuroktribe.nsn.us,False
968,Yurok Tribe of the Yurok Reservation,Redwood Hotel Casino,"P.O. Box 1027 190 Klamath Blvd., Klamath CA 95548",Game Class Type: Class III See 25 CFR 502.3 & 4,True
969,Yurok Tribe of the Yurok Reservation,Redwood Hotel Casino,"P.O. Box 1027 190 Klamath Blvd., Klamath CA 95548",Game Class Type: Class III See 25 CFR 502.3 & 4,True
970,Yurok Tribe of the Yurok Reservation,Redwood Hotel Casino,"P.O. Box 1027 190 Klamath Blvd., Klamath CA 95548",Game Class Type: Class III See 25 CFR 502.3 & 4,True


In [108]:
report_copy = report_df_clean.copy()

In [ ]:
# report_copy.to_csv('../data/temp_gaming_report.csv', index=False)

Manually edit addresses that get combined with Class Type strings, could be in either column 1 or 2.

In [188]:
report_df_clean = pd.read_csv('../data/temp_gaming_report.csv')

In [189]:
# overhang_idx = report_df_clean[1].isna()
is_empty1_idx = np.where(report_df_clean['1'].isna())[0]
# This indicates a place where an address spills over multiple lines
# is_empty1_idx
for i in is_empty1_idx:
    report_df_clean.iloc[i-1, 2] = str(report_df_clean.iloc[i-1, 2]) + ' ' + str(report_df_clean.iloc[i, 2])
    report_df_clean.iloc[i, 2] = np.nan # No longer need the bleed-over text, it has been absorbed into the full name above


In [190]:
report_df_clean.iloc[53:56, :]

,0,1,2,3,is_facility
53,"Bois Forte Band of Minnesota Chippewa Tribe, M...","P.O. Box 16 5344 Lakeshore Drive, Nett Lake MN...",Class Type: Class II & III See 25 CFR 502.3 & 4,Email Address: cchavers@boisforte-nsn.gov,False
54,"Bois Forte Band of Minnesota Chippewa Tribe, M...",Fortune Bay Casino,"P.O. Box 16 5344 Lakeshore Drive, Nett Lake MN...",Game Class Type: Class II & III See 25 CFR 5...,True
55,"Bois Forte Band of Minnesota Chippewa Tribe, M...",NaN,NaN,NaN,True


In [193]:
report_df_clean = report_df_clean.loc[~report_df_clean['1'].isna(), :]

In [ ]:
# report_df_clean.to_csv('../data/clean_gaming_report.csv', index=False)

In [233]:
report_df_clean = pd.read_csv('../data/clean_gaming_report.csv')

In [234]:
# Addresses are in different columns for facilities and non-facilities
report_df_clean.loc[:, 'address'] = np.where(report_df_clean['is_facility'], report_df_clean['2'], report_df_clean['1'])

In [235]:
report_df_clean.loc[:, 'classes_raw'] = np.where(report_df_clean['is_facility'], report_df_clean['3'], report_df_clean['2'])

In [236]:
report_df_clean['facility_name'] = np.where(report_df_clean['is_facility'], report_df_clean['1'], np.nan)

In [237]:
report_df_clean.drop(columns=['1','2','3'], inplace=True)

In [238]:
report_df_clean['state'] = report_df_clean['address'].str.extract(r'([A-Z]{2})')

In [239]:
report_df_clean['classes_raw'].str.contains('Class III')

0      False
1      False
2      False
3      False
4      False
       ...  
857    False
858     True
859     True
860     True
861     True
Name: classes_raw, Length: 862, dtype: object

In [240]:
report_df_clean['classes'] = ''

In [241]:
report_df_clean['classes_raw'].str.contains('Class II & III')

0       True
1       True
2       True
3       True
4       True
       ...  
857     True
858    False
859    False
860    False
861    False
Name: classes_raw, Length: 862, dtype: object

In [242]:
# report_df_clean['classes_raw'].unique()

In [243]:
# report_df_clean.to_csv('../data/raw_gaming_report.csv')
# Save, then manually fix the extra row spaces for:
    # Big Sandy Band of Western Mono Indians
    # Bois Forte
    # Cahuilla Band
    # Campo Band of Diegueno Mission of the Campo Indian Reservation
    # Chicken Ranch Rancheria of Me-wuk Indians of California
    # More?


In [244]:
# report_df_clean = pd.read_csv('../data/raw_gaming_report.csv').iloc[:,1:]

In [245]:
# Since so many state values were on subsequent empty lines, must update after manually cleaning those problem rows
report_df_clean['state'] = report_df_clean['address'].str.extract(r'([A-Z]{2})')

In [246]:
report_df_clean.reset_index(inplace=True, drop=True)

In [247]:
# Split address value by the '\nGame Class' string
# Populate classes_raw
# Fix address value
# report_df_clean.loc[report_df_clean['classes_raw'].isna(),:]#, 'address'].str.find('\n')
classes_raw_isna_idx = report_df_clean['classes_raw'].isna()
report_df_clean.loc[classes_raw_isna_idx, 'classes_raw'] = report_df_clean.loc[classes_raw_isna_idx, 'address']

In [250]:
address_class_idx = report_df_clean.loc[classes_raw_isna_idx, 'address'].str.find('\nGame')

In [263]:
report_df_clean.iloc[437,:]

0                                         Work Phone: 405-379-3321
is_facility                                                   True
address          Email Address: joel.chaffin@creeknationcasino.com
classes_raw      Email Address: joel.chaffin@creeknationcasino.com
facility_name                               Work Fax: 405-379-3504
state                                                          NaN
classes                                                           
Name: 437, dtype: object

OK, so the address is a lost cause at this point. So many of them are merged with Game Class strings, so my script thinks they are in a state abbreviated 'II'. At least that only applies to the script for facilities, since the state for nations should still be accurate. I will just apply a new 'nation_state' column to unify them better, because the revenue still goes back to wherever the nation is based, not where the facility is.

In [269]:
report_df_clean.loc[classes_raw_isna_idx, 'address'] = report_df_clean.loc[classes_raw_isna_idx, 'address'].str.extract(r'^(.*?)\sGame Class')

In [270]:
report_df_clean.loc[:, 'classes'] = 'Other'
report_df_clean.loc[report_df_clean['classes_raw'].str.contains('Class II'), 'classes'] = 'Class II'
report_df_clean.loc[report_df_clean['classes_raw'].str.contains('Class III'), 'classes'] = 'Class III'
report_df_clean.loc[report_df_clean['classes_raw'].str.contains('Class II & III'), 'classes'] = 'Class II & III'



In [272]:
# report_df_clean

In [ ]:
# report_df_clean.to_csv('../data/clean_gaming_report.csv')

The Catawba Indian Nation operates [***one*** Class III facility](https://www.americangaming.org/wp-content/uploads/2025/02/North-Carolina_AGA-Gaming-Regulatory-Fact-Sheet-2025.pdf) in North Carolina. There is a typo in the pdf saying that the facility is in South Carolina, but that is just where the nation is based.

In [274]:
report_df_clean.loc[(report_df_clean.iloc[:,0]=='Catawba Indian Nation') & (~report_df_clean['is_facility']),
                     'classes'] = 'Class III'

In [275]:
report_df_clean.loc[(report_df_clean.iloc[:,0]=='Catawba Indian Nation') & (report_df_clean['is_facility']),
                     'address'] = '538 Kings Mountain Blvd, Kings Mountain, NC 28086'


In [276]:
report_df_clean.loc[(report_df_clean.iloc[:,0]=='Catawba Indian Nation') & (report_df_clean['is_facility']),
                     'state'] = 'NC'


In [277]:
report_df_clean['classes'].value_counts()

classes
Class II & III    574
Class III         142
Class II          120
Other              26
Name: count, dtype: int64

In [278]:
report_df_clean.drop(columns=['classes_raw'], inplace=True)

In [279]:
report_df_clean.rename(columns={'0':'owner_name'}, inplace=True)

In [325]:
owner_name_mapping = report_df_clean.copy().loc[:, ['owner_name', 'state', 'is_facility']]
owner_name_mapping = owner_name_mapping.loc[~owner_name_mapping['is_facility'], ['owner_name', 'state']]
owner_name_mapping.rename(columns={'state':'owner_state'}, inplace=True)


In [337]:
report_df_clean = report_df_clean.merge(owner_name_mapping, on='owner_name')

In [ ]:
# report_df_clean.loc[
#     :, ['facility_name', 'owner_name', 'is_facility', 'address', 'state', 'owner_state', 'classes']
#     ].to_csv('../data/clean/clean_gaming_report.csv', index=False)

### Load from here

In [339]:
import pandas as pd
report_df_clean = pd.read_csv('../data/clean/clean_gaming_report.csv')

In [340]:
owner_names = report_df_clean['owner_name'].unique().tolist()

I really only care about a handful of items from the NIGC gaming report PDF:
* Address of the gaming facility
    * Extract the state as well
* Name of tribe or nation that owns the facility
* Name of the facility
* Class type (Could be a string with some junk in it, will extract Class number later)
* For now, don't really need all the contact info

## Native Economic Trends

In [341]:
import pandas as pd
import difflib

In [342]:
econ = pd.read_excel('../data/native-economic-trends-data_current.xlsx', sheet_name=1)

In [343]:
geonames = econ['geoname_mostrecent'].unique().tolist()

I was considering just using ChatGPT or something to automatically create a crosswalk between the `owner_name` column from the NIGC table and `geoname_mostrecent` here. I should be responsible about validating this crosswalk.

`difflib` should do the trick, too.

In [344]:
best_geonames = []
for owner_name in owner_names:
    best = difflib.get_close_matches(owner_name, geonames, n=1, cutoff=0.46)
    best_geonames.append(best[0] if best else None)

In [345]:
sum([item is None for item in best_geonames]) # Not every owner name had a match!
# Increasing get_close_matches cutoff creates erroneous matches

36

In [346]:
owner_land_crosswalk = pd.DataFrame({'owner_name': owner_names, 'geoname_mostrecent':best_geonames})

In [347]:
owner_land_crosswalk.shape

(258, 2)

It should be noted that this crosswalk does not go between the gaming facility location and reservation location. However, gaming revenues go toward the owners' economic development, which would be centered on the tribe or nation's location, not necessarily just the facilities' locations.

In [348]:
owner_land_crosswalk

,owner_name,geoname_mostrecent
0,Absentee Shawnee Tribe of Indians of OK,None
1,Agua Caliente Band of Cahuilla Indians,None
2,Ak-Chin Indian Communityof the Maricopa Indian...,Prairie Island Indian Community and Off-Reserv...
3,Alabama-Coushatta Tribe of Texas,Alabama-Coushatta Reservation and Off-Reservat...
4,"Alturas Indian Rancheria, California",Alturas Indian Rancheria
...,...,...
253,Yavapai Apache Nation of the Camp Verde Indian...,Yavapai-Apache Nation Reservation
254,Yavapai-Prescott Indian Tribe,Yavapai-Prescott Reservation
255,Yocha DeHe Wintun Nation,Yomba Reservation
256,Ysleta del Sur Pueblo,Isleta Pueblo


In [349]:
owner_land_crosswalk['owner_name'].unique().size

258

In [350]:
report_df_clean['owner_name'].unique().size

258

In [351]:
owner_land_crosswalk['geoname_mostrecent'].unique().size

131

In [352]:
econ['geoname_mostrecent'].unique().size

771

In [ ]:
# owner_land_crosswalk.to_csv('../data/unverified_crosswalk.csv', index=False)

I will manually edit this crosswalk csv to ensure accuracy. To clarify, there will be a one-to-many mapping from nations to reservations that are shared by multiple nations. For example, "Citizen Potawatomi Nation-Absentee Shawnee OTSA" belongs to both "Absentee Shawnee Tribe of Indians of OK" and Citizen Potawatomi.

In [359]:
owner_land_crosswalk = pd.read_csv('../data/clean/verified_crosswalk.csv')
owner_land_crosswalk

,owner_name,geoname_mostrecent,no_reservation_or_not_listed
0,Absentee Shawnee Tribe of Indians of OK,Citizen Potawatomi Nation-Absentee Shawnee OTSA,NaN
1,Agua Caliente Band of Cahuilla Indians,Agua Caliente Indian Reservation and Off-Reser...,NaN
2,Ak-Chin Indian Communityof the Maricopa Indian...,Maricopa (Ak Chin) Indian Reservation and Off-...,NaN
3,Alabama-Coushatta Tribe of Texas,Alabama-Coushatta Reservation and Off-Reservat...,NaN
4,"Alturas Indian Rancheria, California",Alturas Indian Rancheria,NaN
...,...,...,...
236,Yavapai Apache Nation of the Camp Verde Indian...,Yavapai-Apache Nation Reservation,NaN
237,Yavapai-Prescott Indian Tribe,Yavapai-Prescott Reservation,NaN
238,Yocha DeHe Wintun Nation,Rumsey Indian Rancheria,NaN
239,Ysleta del Sur Pueblo,Ysleta del Sur Pueblo and Off-Reservation Trus...,NaN


In [361]:
report_df_clean.loc[report_df_clean['owner_name']=='Morongo Band of Mission Indians, California', :]

,facility_name,owner_name,is_facility,address,state,owner_state,classes


In [362]:
report_df_clean.loc[report_df_clean['facility_name']=='Casino Morongo', :]

,facility_name,owner_name,is_facility,address,state,owner_state,classes


In [366]:
report_df_clean.dtypes

facility_name    object
owner_name       object
is_facility        bool
address          object
state            object
owner_state      object
classes          object
dtype: object

In [370]:
facility_name = [np.nan, 'Morongo Travel Center @ Morongo Trail', 'Casino Morongo', 'Morongo Casino Resort & Spa', 'Morongo Travel Center @ Seminole Dr.', 'Morongo Canyon Lanes']
owner_name = ['Morongo Band of Mission Indians, California'] * len(facility_name)
is_facility = [False, True, True, True, True, True]
address = ['12700 Pumarra Road, Banning CA 92220'] * len(facility_name)
state = ['CA'] * len(facility_name)
owner_state = state
classes = ['Class II & III'] * 4 + ['Class II'] * 2
report_snippet_fix = pd.DataFrame({'facility_name':facility_name,
                                   'owner_name':owner_name,
                                   'is_facility':is_facility,
                                   'address':address,
                                   'state':state,
                                   'owner_state':owner_state,
                                   'classes':classes})

In [372]:
report_df_clean_final = pd.concat([report_df_clean, report_snippet_fix], axis=0)

In [373]:
report_df_clean_final.reset_index(drop=True, inplace=True)

In [ ]:
# report_df_clean_final.to_csv('../data/clean/clean_gaming_report.csv', index=False)